# Intelligent Realestate Investing 

In [1]:
#https://rapidapi.com/apidojo/api/realtor
import pandas as pd
import requests

In [118]:
# Function using the realator api to grabe houses that are for sale!!

def api_property_list_for_sale(city, state, prop_type, limit=200):

    url = "https://realtor.p.rapidapi.com/properties/v2/list-for-sale"

    querystring = {"city":city,"limit":"200","offset":"0","state_code":state,"sort":"relevance","prop_type":prop_type}

    headers = {
        'x-rapidapi-key': "3098984b99msh9a120a8ade10e53p1ebdd1jsn7703d6e155d0",
        'x-rapidapi-host': "realtor.p.rapidapi.com"
        }

    response = requests.request("GET", url, headers=headers, params=querystring)

    return response.json() # json format

In [119]:
# Once pick state and home type throw it through this function to get the proper readable format
def process_list_for_sale_response(response_json):
    """
    Process the list for sale API response.

    Convert each listing to a dataframe, append to a list, and concatenate to one dataframe.

    Parameters
    ----------
    @response_json [dictionary]: API response for list for sale

    Returns
    -------
    [dataframe] Dataframe of all list for sale responses

    """

    # empty dataframe
    dataframe_list = []

    # iterate through each for sale listing
    for l in response_json['properties']:

        # convert each listing to dataframe
        _temp_df = pd.DataFrame.from_dict(l, orient='index').T

        # append to dataframe list for all listings
        dataframe_list.append(_temp_df)

    # concatenate all dataframes, for missing col values enter null value
    return pd.concat(dataframe_list, axis=0, ignore_index=True, sort=False)

In [120]:
def make_house_data_clean(df_properties_for_sale_raw):
    df_properties_for_sale_raw = pd.DataFrame(df_properties_for_sale_raw)# transfer into dataframe
    df_pol_ps = df_properties_for_sale_raw['lot_size'].apply(pd.Series) # make this int column non interable into series then 
    df_properties_for_sale_raw['size'] = df_pol_ps['size'] # then make it into column 

    # pick out only the columns we want  and rename them 
    list2 = ['property_id', 'listing_id','address','prop_status','price', 'baths_full', 'baths', 'beds', 'building_size','last_update', 'lot_size','baths_half', 'size']
    result = df_properties_for_sale_raw[list2]
    result.columns = ['property_id', 'listing_id','address','prop_status','price', 'baths_full', 'baths', 'beds', 'building_size','last_update', 'lot_size','baths_half', 'lot_size']
    return result

In [121]:
def get_address_and_buildingsize(result):
    address = pd.json_normalize(result['address'])
    building_size = pd.json_normalize(result['building_size'])
    building_size.columns =['building_size','building_inside_units']
    result2 = pd.concat([address,building_size], axis=1)
    result3 = pd.concat([df,result2], axis=1)
    return result3


In [122]:
# Variables 
city = "myrtle beach"
state = "SC"
prop_type = "single_family"

property_list_for_sale_response = api_property_list_for_sale(
                                                             city=city, 
                                                             state=state, 
                                                             prop_type=prop_type,
                                                             limit=200)

In [123]:
df_properties_for_sale_raw = process_list_for_sale_response(response_json=property_list_for_sale_response)


In [124]:
df = make_house_data_clean(df_properties_for_sale_raw)
#list(df.columns)
df

,property_id,listing_id,address,prop_status,price,baths_full,baths,beds,building_size,last_update,lot_size,baths_half,lot_size
0,M9090577838,2929605773,"{'city': 'Myrtle Beach', 'line': '2185 Sanibel...",for_sale,424900,3,3,4,"{'size': 1725, 'units': 'sqft'}",2021-05-22T04:15:29Z,NaN,NaN,NaN
1,M9155203410,2929592057,"{'city': 'Myrtle Beach', 'line': '9035 Gatewic...",for_sale,249900,2,2,3,"{'size': 1394, 'units': 'sqft'}",2021-05-21T20:51:17Z,"{'size': 7841, 'units': 'sqft'}",NaN,7841.0
2,M9416395684,2929590343,"{'city': 'Myrtle Beach', 'line': '650 Elgin Ct...",for_sale,394900,3,4,4,"{'size': 2624, 'units': 'sqft'}",2021-05-22T00:31:43Z,"{'size': 15682, 'units': 'sqft'}",1,15682.0
3,M5723306696,2929587613,"{'city': 'Myrtle Beach', 'line': '2534 Goldfin...",for_sale,400000,2,2,3,"{'size': 2148, 'units': 'sqft'}",2021-05-21T19:44:11Z,"{'size': 13068, 'units': 'sqft'}",NaN,13068.0
4,M9372246768,2929586258,"{'city': 'Myrtle Beach', 'line': '6303 Porcher...",for_sale,569000,3,3,4,"{'size': 1929, 'units': 'sqft'}",2021-05-22T14:10:22Z,"{'size': 11326, 'units': 'sqft'}",NaN,11326.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,M9549522450,2929083084,"{'city': 'Myrtle Beach', 'line': '2653 Corn Pi...",for_sale,249000,2,2,3,"{'size': 1464, 'units': 'sqft'}",2021-05-10T21:58:38Z,"{'size': 19602, 'units': 'sqft'}",NaN,19602.0
196,M9698682059,2929082893,"{'city': 'Myrtle Beach', 'line': '6001-1454 S ...",for_sale,237000,3,3,4,"{'size': 1610, 'units': 'sqft'}",2021-05-17T15:11:36Z,"{'size': 1742, 'units': 'sqft'}",NaN,1742.0
197,M9517132030,2929080051,"{'city': 'Myrtle Beach', 'line': '3556 Lightho...",for_sale,198500,2,2,2,"{'size': 1076, 'units': 'sqft'}",2021-05-09T18:04:01Z,"{'size': 8712, 'units': 'sqft'}",NaN,8712.0
198,M9903420168,2929075197,"{'city': 'Myrtle Beach', 'line': '289 Wateree ...",for_sale,289900,2,2,3,"{'size': 2229, 'units': 'sqft'}",2021-05-10T20:49:28Z,"{'size': 10454, 'units': 'sqft'}",NaN,10454.0


In [125]:
# expand the address and building size columns 
var2 = get_address_and_buildingsize(df)
var2['line']

0              2185 Sanibel Ct
1             9035 Gatewick Ct
2                 650 Elgin Ct
3            2534 Goldfinch Dr
4              6303 Porcher Dr
                ...           
195          2653 Corn Pile Rd
196    6001-1454 S Kings Hwy S
197        3556 Lighthouse Way
198       289 Wateree River Rd
199        4351 Red Rooster Ln
Name: line, Length: 200, dtype: object

In [126]:
var2

,property_id,listing_id,address,prop_status,price,baths_full,baths,beds,building_size,last_update,...,county,fips_code,county_needed_for_uniq,time_zone,lat,lon,neighborhood_name,is_approximate,building_size,building_inside_units
0,M9090577838,2929605773,"{'city': 'Myrtle Beach', 'line': '2185 Sanibel...",for_sale,424900,3,3,4,"{'size': 1725, 'units': 'sqft'}",2021-05-22T04:15:29Z,...,Horry,45051,False,America/New_York,33.669617,-78.908846,South Myrtle Beach,NaN,1725,sqft
1,M9155203410,2929592057,"{'city': 'Myrtle Beach', 'line': '9035 Gatewic...",for_sale,249900,2,2,3,"{'size': 1394, 'units': 'sqft'}",2021-05-21T20:51:17Z,...,Horry,45051,False,NaN,33.749447,-78.939917,Carolina Forest,NaN,1394,sqft
2,M9416395684,2929590343,"{'city': 'Myrtle Beach', 'line': '650 Elgin Ct...",for_sale,394900,3,4,4,"{'size': 2624, 'units': 'sqft'}",2021-05-22T00:31:43Z,...,Horry,45051,False,NaN,33.741506,-78.909741,Carolina Forest,NaN,2624,sqft
3,M5723306696,2929587613,"{'city': 'Myrtle Beach', 'line': '2534 Goldfin...",for_sale,400000,2,2,3,"{'size': 2148, 'units': 'sqft'}",2021-05-21T19:44:11Z,...,Horry,45051,False,America/New_York,33.670222,-78.967307,NaN,NaN,2148,sqft
4,M9372246768,2929586258,"{'city': 'Myrtle Beach', 'line': '6303 Porcher...",for_sale,569000,3,3,4,"{'size': 1929, 'units': 'sqft'}",2021-05-22T14:10:22Z,...,Horry,45051,False,NaN,33.736093,-78.829219,NaN,NaN,1929,sqft
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,M9549522450,2929083084,"{'city': 'Myrtle Beach', 'line': '2653 Corn Pi...",for_sale,249000,2,2,3,"{'size': 1464, 'units': 'sqft'}",2021-05-10T21:58:38Z,...,Horry,45051,False,NaN,33.723513,-78.988895,Forestbrook,NaN,1464,sqft
196,M9698682059,2929082893,"{'city': 'Myrtle Beach', 'line': '6001-1454 S ...",for_sale,237000,3,3,4,"{'size': 1610, 'units': 'sqft'}",2021-05-17T15:11:36Z,...,Horry,45051,False,NaN,33.626229,-78.960292,Ocean Lakes,NaN,1610,sqft
197,M9517132030,2929080051,"{'city': 'Myrtle Beach', 'line': '3556 Lightho...",for_sale,198500,2,2,2,"{'size': 1076, 'units': 'sqft'}",2021-05-09T18:04:01Z,...,Horry,45051,False,America/New_York,33.727377,-78.876152,Myrtlewood,NaN,1076,sqft
198,M9903420168,2929075197,"{'city': 'Myrtle Beach', 'line': '289 Wateree ...",for_sale,289900,2,2,3,"{'size': 2229, 'units': 'sqft'}",2021-05-10T20:49:28Z,...,Horry,45051,False,NaN,33.618690,-79.020027,The Lakes,NaN,2229,sqft


# Now remove all of the numbers for the address and get zestimate and put them in to melissa .com for the other addresses on the street